In [5]:
%matplotlib inline

import numpy as np
from matplotlib import pyplot as plt
from numpy import random as rnd
import warnings,datetime,os,calendar,csv,time

import pickle,h5py,json

import tensorflow as tf
import pandas as pd
import seaborn as sns

from keras.models import Model,Sequential
from keras.layers import Dense,LSTM,Conv2D,Dropout,BatchNormalization,Input,Concatenate,Add,Activation,MaxPooling2D,AveragePooling2D
import keras.backend as K

from sklearn import preprocessing as pp
from sklearn.cluster import KMeans,DBSCAN,MeanShift,AgglomerativeClustering
from sklearn.mixture import GaussianMixture

from statsmodels.tsa.seasonal import seasonal_decompose

import pyodbc

sns.set()
warnings.filterwarnings('ignore')

In [21]:
dbpath = '..\\..\\..\\..\\banking.accdb'

conn = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ='+dbpath+';')
# Read Single Transactions table
STR = pd.read_sql_query(
'''
SELECT * FROM STR
'''
,conn).set_index('STR_id')
# Close connection
conn.close()
# Transform STR
STR['Date_booked'] = pd.to_datetime(STR['Date_booked'])
STR['Date_ordered'] = pd.to_datetime(STR['Date_ordered'])
#STR['Value_transaction'] = STR['Value_transaction'].astype('float')
# Select rows with unknown single customer ID
STR = STR[STR['SCA_id']=='None']

In [22]:
rf = STR[STR['Value_transaction']>0]
nf = STR[STR['Value_transaction']==0]
ef = STR[STR['Value_transaction']<0]

TypeError: '>' not supported between instances of 'str' and 'int'

In [24]:
STR[['Value_transaction']]

,Value_transaction
STR_id,
346,-50.0
347,0.0
348,-4.5
349,-235.0
350,-24.9
351,-66.89
352,-64.9
353,-62.0
354,-2.05
